# `ecco` modelling language

In this notebook we see how to model an ecosystem using the _reaction rules_ language, aka `rr`.
As an example, we consider the model of a termites colony that is a variation of that presented in [DOI:10.1111/2041-210X.13242](https://doi.org/10.1111/2041-210X.13242).
Below we use a ready made model that one can find in [`termites.rr`](termites.rr) in `doc` directory along with this notebook. This file may be edited directly, but instead we load it into `ecco` in order to examine its content and show its graphical representations.

## Start `ecco` and load a `rr` model

First of all, we start `ecco`, passing to it the name of the `rr` file we want to load:

In [1]:
%run -m ecco termites.rr

This creates an object `model` that deals with the static aspects of the `rr` model.
To display help about this object, type `model?` in a code cell, or `model.method?` to display the help about a specific method.

## Displaying the model

Property `model.rr` displays the `rr` source code:

In [2]:
model.rr

HTML(value="<pre style='line-height:140%'><span style='color:#008; font-weight:bold;'>inhabitants:</span>\n   …

A model consists of a series of sections, each has a name followed by a colon and an indented block that is its content. The first sections have freely chosen names (above `inhabitants`, `structures`, `resources`, and `competitors`) and define the variables of the models, the two last sections (`constraints` and `rules`) define the evolution rules of the model.

### Comments

Character `#` introduces a comment that extends until the end of the line. When a model is displayed by `ecco` as above, a comment is added to give a name to every rule orconstraint as shown above.

### Variables

A model has Boolean variables, each of which having a name, an initial state, and a description. For instance, `Rp+: reproductive` defines a variable called `Rp`, whose initial state is on (`+`), and whose description is `reproductives`. Similarly, `Wk-: workers` defines a variable called `Wk`, whose initial state is off (`-`), and whose description is `workers`. Separating the variables definitions into sections and providing descriptions has no consequences on the semantics but may be useful to clarify the model.

### Evolution rules

Each rule or constraint describes a possible evolution in the system and consists of a left-hand side that is a condition for its execution, and a right-hand side that is an assignement of some variables to be done upon the execution of the rule or constraint. For instance, rule `Ac+, Sd- >> Wk-, Rp-` has left-hand side `Ac+, Sd-` that states that the rule may be executed when variable `Ac` is on and variable `Sd` is off, and a right-had side `Wk-, Rp-` that states that if the rule is executed, then `Wk` and `Rp` are both set to off (regardless of their previous state). This could be read as _when ant competitors are present and no soldier is present, workers and reproductives may be killed_, but this could be as well interpreted as _when too much ant competitors are present and not enough soldiers are present, then most workers and reproductives may be killed_ which is a more quantitative interpretation of the Boolean setting.

### Constraints vs rules

The only difference between constraints and rules is that the former have a higher priority than the latter: if a constraint can be executed then no rule is allowed to be executed, or in other words, a rule can be executed only if no constraint can. A model may have no constraints in which case the corresponding section is completely omitted. Constraints are usually used to model quick or mandatory processes in an ecosystem, typically, when then place where some species live disappear, then the species have to disappear has well (eg, fishes in a pond will die if the pond dries). In our termites example, termitomicies cannot exist without a fungal garden, hence the constraint.

## Graphical representations of a `rr` model

Method `ecograph` displays the _ecosystemic graph_ (that is, an influence graph whose nodes are the variables and whose edges represent the influence of one node onto another). Try to type `model.ecograph?` to discover the many available options.

Graphs are interactive:
 * clicking on nodes selects them and displays their attached information in the _Inspector_ tab below the graph
 * various tabs with controls allow to change visual attributes
   * _Graph_ for the graph layout
   * _Nodes_ for the appearance of nodes (colors, shapes, etc.)
   * _Edges_ for the appearance of edges (colors, tips, etc.)
 * _Export_ tab allow to get the graph as PDF, LaTeX, etc., file. This feature is so far exerimental and requires the user to help capturing the nodes positions by moving the mouse over them.

In [3]:
model.ecograph()

Method `model.ecohyper` displays the _ecosystemic hypergraph_ that is another graphical representation of the `rr` model. This is a more precise representation that actually loses only the description of the variables and their dispatching into sections (these informations are actually only for the modeller but carry no semantical meaning). In this representation, round-shaped nodes are the variables, and square-shaped nodes are the rules and constraints (the latter being displayed with a yellow background). An edge links a rule/constraint with each variables that is used in it, a white (resp. black) dot represents a sign `-` (resp. `+`), and the left-hand side of the rule corresponds to the rule-side of the edge, while the right-hand side corresponds to the variable-side of the edge. See for instance how rule R9, that is `Ac+, Sd- >> Wk-, Rp-`, is rendered.

In [4]:
model.ecohyper()

## Semantics

A rule or constraint may be executed from a state `s` if its left-hand side is validated by `s`. For example, in the initial state of our termites model (that could be written as `{Rp+, Ac+}`, all the other variables being off), only rules `Rp+ >> Ec+` and `Ac+, Sd- >> Wk-, Rp-` may be executed. Executing a rule or constraint yields a state that is obtained by assigning the variables as instructed by the right-hand side of the executed rule of constraint. For instance, executing `Rp+ >> Ec+` from the initial state `{Rp+, Ac+}` yields state `{Rp+, Ac+, Ec+}`, while executing `Ac+, Sd- >> Wk-, Rp-` from the same state yields state `{Ac+}`.

The semantics of a `rr` model is a state-space, that is a graph whose nodes are the states that can be reached by executing rules or constraints, and whose edges are labelled with the rules or constraints allowing to reach one state from another. It can be constructed as follows:

 * the initial state, as described in the `rr` model, is a node of state-space
 * if `s` is a state and `c` is a constraint that can be executed from `s` yielding a new state `s'` distinct from `s`, then there is an edge `s -> s'` labelled by `c` in the state-space
 * if `s` is a state such that no constraint can be executed from it, and if `r` is a rule that can be executed from `s` yielding a new state `s'` distinct from `s`, then there is an edge `s -> s'` labelled by `r` in the state-space

Note that above we required `s` to be distinct from `s'` and thus we have no edges `s -> s` in a state-space (so called side-loops).

The semantics is fully detailled in [DOI:10.1007/978-3-031-06653-5_10](https://doi.org/10.1007/978-3-031-06653-5_10).

### Transient states

A state that enables a constraint is called _transient_, it can be seen as a state that the ecosystem can reach but only for a short period of time. In a state-space analysis, transient states may be kept or removed, depending on what one wants to observe.